# Wan2.2 Video Generation on Google Colab

Wan2.2をGoogle Colabで動かすためのノートブック

## 特徴
- **Google Driveにモデル保存** - 毎回ダウンロード不要
- **複数モデル対応** - 用途に応じて選択可能
- **GGUF量子化対応** - T4でも14Bモデル動作可能
- **カスタムノード対応** - WanVideoWrapper, GGUF, VideoHelperSuite, QwenEditUtils

## 対応モデル

### Wan2.2 Diffusion Models（モジュラー型・動画生成）
| モデル | サイズ | VRAM目安 | 用途 |
|--------|--------|----------|------|
| I2V-14B-fp16 | 28.6GB | 24GB+ | 高品質I2V |
| I2V-14B-fp8 | 14.3GB | ~16GB | 中VRAM I2V |
| I2V-14B-GGUF-Q4 | ~8GB | ~12GB | T4対応I2V |
| T2V-14B-fp16 | 28.6GB | 24GB+ | 高品質T2V |
| T2V-14B-fp8 | 14.3GB | ~16GB | 中VRAM T2V |
| TI2V-5B | ~10GB | ~8GB | 軽量版（テキスト+画像） |
| Animate-14B-GGUF-Q4 | ~8GB | ~12GB | アニメーション |

### Qwen Image Edit（チェックポイント型・画像編集）
| モデル | サイズ | VRAM目安 | 用途 |
|--------|--------|----------|------|
| Qwen-Rapid-AIO-NSFW-v18 | 28.4GB | 24GB+ | 画像編集・生成 |

### High Noise vs Low Noise (Wan)
- **High Noise**: 大きな動き・変化を生成（推奨）
- **Low Noise**: 入力により忠実、変化控えめ

## ワークフロー
| ファイル | 用途 | 必要モデル |
|----------|------|------------|
| t2v-14b.json | テキスト→動画 | T2V-14B + TextEncoder + VAE |
| i2v-14b.json | 画像→動画 | I2V-14B + TextEncoder + VAE + CLIP Vision |
| low-vram-ti2v.json | T4対応TI2V | TI2V-5B + LoRA + TextEncoder + VAE |
| qwen-image-edit.json | 画像編集 | Qwen-Rapid-AIO-v18 |

---
## 1. GPU確認

In [ ]:
import torch

!nvidia-smi
print("\n" + "="*50)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    if gpu_memory >= 40:
        print("推奨: fp16モデル (I2V-14B, T2V-14B)")
    elif gpu_memory >= 20:
        print("推奨: fp16またはGGUF-Q8")
    else:
        print("推奨: GGUF-Q4量子化モデル or TI2V-5B")
else:
    print("GPU not available!")
print("="*50)

---
## 2. Google Driveマウント

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# モデル保存用ディレクトリ
DRIVE_MODEL_DIR = "/content/drive/MyDrive/ComfyUI_Wan22"

# フォルダ構成作成
folders = [
    f"{DRIVE_MODEL_DIR}/diffusion_models",
    f"{DRIVE_MODEL_DIR}/text_encoders",
    f"{DRIVE_MODEL_DIR}/vae",
    f"{DRIVE_MODEL_DIR}/clip_vision",
    f"{DRIVE_MODEL_DIR}/loras",
    f"{DRIVE_MODEL_DIR}/checkpoints",
    f"{DRIVE_MODEL_DIR}/workflows",
    f"{DRIVE_MODEL_DIR}/outputs"
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print(f"モデル保存先: {DRIVE_MODEL_DIR}")

# GitHubからワークフローをダウンロード
GITHUB_RAW = "https://raw.githubusercontent.com/hatoya0703/Colab.ComfyUI/main/workflows"
workflows = [
    "t2v-14b.json",
    "i2v-14b.json",
    "low-vram-ti2v.json",
    "qwen-image-edit.json"
]
print("\nワークフローをGitHubからダウンロード:")
for wf in workflows:
    dest = f"{DRIVE_MODEL_DIR}/workflows/{wf}"
    if not os.path.exists(dest):
        !wget -q -O "{dest}" "{GITHUB_RAW}/{wf}"
        print(f"  DL: {wf}")
    else:
        print(f"  既存: {wf}")

print("\n既存のモデル:")
!ls -la "{DRIVE_MODEL_DIR}/diffusion_models/" 2>/dev/null || echo "(なし)"
!ls -la "{DRIVE_MODEL_DIR}/text_encoders/" 2>/dev/null || echo "(なし)"
!ls -la "{DRIVE_MODEL_DIR}/checkpoints/" 2>/dev/null || echo "(なし)"
print("\n既存のワークフロー:")
!ls -la "{DRIVE_MODEL_DIR}/workflows/" 2>/dev/null || echo "(なし)"

---
## 3. ComfyUI + カスタムノード インストール

In [ ]:
%%time
import os

# ComfyUI
if not os.path.exists("/content/ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!pip install -q huggingface_hub
print("ComfyUI installed.")

# Kijai WanVideoWrapper
if not os.path.exists("custom_nodes/ComfyUI-WanVideoWrapper"):
    !git clone https://github.com/kijai/ComfyUI-WanVideoWrapper.git custom_nodes/ComfyUI-WanVideoWrapper
    !pip install -q -r custom_nodes/ComfyUI-WanVideoWrapper/requirements.txt
print("WanVideoWrapper installed.")

# GGUF support
if not os.path.exists("custom_nodes/ComfyUI-GGUF"):
    !git clone https://github.com/city96/ComfyUI-GGUF.git custom_nodes/ComfyUI-GGUF
    !pip install -q gguf
print("GGUF support installed.")

# VideoHelperSuite
if not os.path.exists("custom_nodes/ComfyUI-VideoHelperSuite"):
    !git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git custom_nodes/ComfyUI-VideoHelperSuite
    !pip install -q -r custom_nodes/ComfyUI-VideoHelperSuite/requirements.txt 2>/dev/null || true
print("VideoHelperSuite installed.")

# KJNodes (for Low-VRAM TI2V workflow)
if not os.path.exists("custom_nodes/ComfyUI-KJNodes"):
    !git clone https://github.com/kijai/ComfyUI-KJNodes.git custom_nodes/ComfyUI-KJNodes
    !pip install -q -r custom_nodes/ComfyUI-KJNodes/requirements.txt 2>/dev/null || true
print("KJNodes installed.")

# rgthree-comfy (for Low-VRAM TI2V workflow)
if not os.path.exists("custom_nodes/rgthree-comfy"):
    !git clone https://github.com/rgthree/rgthree-comfy.git custom_nodes/rgthree-comfy
print("rgthree-comfy installed.")

# Qwen Image Edit Utils (for Qwen Image Edit workflow)
if not os.path.exists("custom_nodes/Comfyui-QwenEditUtils"):
    !git clone https://github.com/lrzjason/Comfyui-QwenEditUtils.git custom_nodes/Comfyui-QwenEditUtils
print("QwenEditUtils installed.")

# SageAttention (for Low-VRAM optimization)
!pip install -q sageattention
print("SageAttention installed.")

print("\nAll custom nodes installed!")

---
## 4. モデルをシンボリックリンクで接続

In [ ]:
import os

os.chdir("/content/ComfyUI")

# 既存のmodelsフォルダを削除
!rm -rf models/diffusion_models models/text_encoders models/vae models/clip_vision models/loras models/checkpoints 2>/dev/null

# シンボリックリンク作成（モデル）
links = [
    (f"{DRIVE_MODEL_DIR}/diffusion_models", "models/diffusion_models"),
    (f"{DRIVE_MODEL_DIR}/text_encoders", "models/text_encoders"),
    (f"{DRIVE_MODEL_DIR}/vae", "models/vae"),
    (f"{DRIVE_MODEL_DIR}/clip_vision", "models/clip_vision"),
    (f"{DRIVE_MODEL_DIR}/loras", "models/loras"),
    (f"{DRIVE_MODEL_DIR}/checkpoints", "models/checkpoints"),
]

for src, dst in links:
    if os.path.exists(src):
        !ln -sf "{src}" "{dst}"
        print(f"リンク: {dst} -> {src}")

# ワークフローのシンボリックリンク
os.makedirs("user/default", exist_ok=True)
!rm -rf user/default/workflows 2>/dev/null
if os.path.exists(f"{DRIVE_MODEL_DIR}/workflows"):
    !ln -sf "{DRIVE_MODEL_DIR}/workflows" "user/default/workflows"
    print(f"リンク: user/default/workflows -> {DRIVE_MODEL_DIR}/workflows")

print("\n=== モデル確認 ===")
!ls -la models/diffusion_models/
!ls -la models/text_encoders/
!ls -la models/vae/
!ls -la models/clip_vision/
!ls -la models/checkpoints/
print("\n=== ワークフロー確認 ===")
!ls -la user/default/workflows/ 2>/dev/null || echo "workflows: (なし)"

---
## 5. ComfyUI起動

In [ ]:
import os
os.chdir("/content/ComfyUI")

# Colab SecretsからNGROKトークンを取得
from google.colab import userdata
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
except userdata.SecretNotFoundError:
    NGROK_TOKEN = None
    print("⚠️ NGROK_TOKEN が Colab Secrets に設定されていません")
    print("  左サイドバー 🔑 → 新しいシークレットを追加 → NGROK_TOKEN")

# ngrokトンネル
if NGROK_TOKEN:
    !pip install -q pyngrok
    from pyngrok import ngrok
    ngrok.kill()  # 既存のトンネルを停止
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(8188)
    print(f"\n{'='*50}")
    print(f"ComfyUI URL: {public_url}")
    print(f"{'='*50}\n")
else:
    print("="*50)
    print("ngrok未設定")
    print("Colabの出力ポートからアクセスしてください")
    print("="*50)

# ComfyUI起動
!python main.py --listen 0.0.0.0 --port 8188

---
## Tips

### 起動手順
1. GPU確認
2. Google Driveマウント
3. ComfyUIインストール
4. シンボリックリンク
5. ComfyUI起動

### GPU別推奨モデル
| GPU | VRAM | 推奨モデル |
|-----|------|------------|
| T4 | 15GB | GGUF-Q4, TI2V-5B, fp8 |
| L4 | 24GB | fp16, fp8, Qwen |
| A100 | 40GB | fp16, Qwen |

### フォルダ構成
```
Google Drive/MyDrive/ComfyUI_Wan22/
├── diffusion_models/  # Wan2.2モデル（モジュラー型）
├── text_encoders/     # テキストエンコーダー
├── vae/               # VAE
├── clip_vision/       # CLIP Vision
├── loras/             # LoRA
├── checkpoints/       # Qwenモデル（チェックポイント型）
├── workflows/         # ワークフロー（自動読み込み）
└── outputs/           # 出力動画・画像
```

### Wan 14B推奨設定
- Sampler: `uni_pc`
- Scheduler: `simple`
- CFG: `5`
- Steps: `30`
- ModelSamplingSD3 shift: `3.0`

### Qwen Image Edit v18推奨設定
- Sampler: `euler_ancestral`
- Scheduler: `beta`
- CFG: `1`
- Steps: `4`